In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import glob

In [ ]:
base_path  = Path('./EPA_GHGI_2025/2025_Main_Report_Tables') #draft 2025 report (1990-2023)
# dataset = 'EPA_GHGI_DataExplorer/Data_Explorer_output.csv' #dataexplorer only goes to the 2024 report 1990-2022

In [ ]:
sectors_final = ['Energy', 'Manufacturing', 'Land Management', 'Waste Management']

In [ ]:
sectors_EPA = ['Energy', 'Industrial Processes and Product Use', 'Agriculture', 'Waste', 'Total Gross Emissionsd  (Sources)', 
'LULUCF Sector Net Totae', 'Net Emission (Sources and Sinks)f']
## [SIC] TYPOS IN THE DRAFT REPORT TABLES

## Functions

In [ ]:
def parse_report_tables(glob_path, notes_start_row):
    path = list(base_path.glob(glob_path))[0]

    # Notes and title
    meta = pd.read_csv(path)
    title = meta.columns[0]
    notes = meta.iloc[notes_start_row:,0].tolist()

    # Data
    df_clean = pd.read_csv(path, header=1)
    df_clean.set_index(df_clean.columns[0],inplace=True)
    df_clean = df_clean.iloc[:notes_start_row-1,:]
    return title, notes, df_clean

## Emission Types for Each Sector

### Import data and parse

In [ ]:
emit_glob = "Chapter 2*/Table 2-3.csv"
title_emit, notes_emit, df_emit = parse_report_tables(emit_glob, 64)
print(title_emit,*notes_emit, sep='\n')

In [ ]:
# df_emit

In [ ]:
# WHATEVER VERSION OF THESE TABLES ARE THE DRAFTS NOT THE FINAL REPORT TABLES! TYPOS!!!!!!
# df_clean.index.tolist() 

In [ ]:
# Need to split by subheaders:
sector_col = []
current_sector = None

for idx, row in df_emit.iterrows():
    if idx in sectors_EPA:
        current_sector = idx
        sector_col.append(current_sector)
    else:
        sector_col.append(current_sector)

df_emit['Sector'] = sector_col
new_columns = pd.MultiIndex.from_arrays([df_emit['Sector'], df_emit.index.get_level_values(0)])
# new_columns[0]

df_emit.set_index(new_columns, inplace=True)

# Select 2021-2023
df_emit_sub = df_emit.iloc[:, -4:-1]
df_emit_sub

In [ ]:
#Check the typos were all resolved
# df_emit_sub.index.values

### Reorganize into sectors_final

In [ ]:
sectors_final

In [ ]:
# Energy
df_E = df_emit_sub.loc[sectors_EPA[0]].drop([sectors_EPA[0]])
# df_E["2021-2023 Average"] = df_E.mean(axis=1)

# Manufacturing
df_M = df_emit_sub.loc[sectors_EPA[1]].drop([sectors_EPA[1]])
# df_M["2021-2023 Average"] = df_M.mean(axis=1)

# Land mgmt
df_L1 = df_emit_sub.loc[sectors_EPA[2]].drop([sectors_EPA[2]])
df_L2 = df_emit_sub.loc[sectors_EPA[5]].drop([sectors_EPA[5]])
df_L = pd.concat([df_L1, df_L2])
# df_L["2021-2023 Average"] = df_L.mean(axis=1)


# Waste mgmt
df_W = df_emit_sub.loc[sectors_EPA[3]].drop([sectors_EPA[3]])
# df_W["2021-2023 Average"] = df_W.mean(axis=1)


# Store dataframes in a dictionary
dataframes_emittype = {
    sectors_final[0]: df_E,
    sectors_final[1]: df_M,
    sectors_final[2]: df_L,
    sectors_final[3]: df_W,
    
}

### Calculate average and percents and write out

In [ ]:
# Write to csv:
for key, df in dataframes_emittype.items():
    df["2021-2023 Average"] = df.mean(axis=1)
    if key != sectors_final[2]:
        df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100 # Calculate percent for all but land management
    output=f'output_tables/{key}_Sectors.csv'
    df.to_csv(output)

## Emission Gas Type for Each Sector

### Energy

In [ ]:
energy_gas_glob = "Chapter 2*/Table 2-4.csv"
title_E_gas, notes_E_gas, df_E_gas = parse_report_tables(energy_gas_glob, 39)
print(title_E_gas,*notes_E_gas, sep='\n')

In [ ]:
gas_energy = ['CO2','CH4', 'N2O']
df_E_gas_sub = df_E_gas.loc[gas_energy].iloc[:, -4:-1]
df_E_gas_sub

### Manufacturing

In [ ]:
man_gas_glob = "Chapter 2*/Table 2-6.csv"
title_M_gas, notes_M_gas, df_M_gas = parse_report_tables(man_gas_glob, 58)
print(title_M_gas,*notes_M_gas, sep='\n')

In [ ]:
gas_man = ['CO2','CH4', 'N2O', 'HFCs', 'PFCs', 'SF6', 'NF3']
df_M_gas_sub = df_M_gas.loc[gas_man].iloc[:, -4:-1]
df_M_gas_sub

### Land Management (this needs work for the land use/change table - how do I extract gases)

### Agriculture

In [ ]:
land1_gas_glob = "Chapter 2*/Table 2-7.csv"
title_L1_gas, notes_L1_gas, df_L1_gas = parse_report_tables(land1_gas_glob, 14)
print(title_L1_gas,*notes_L1_gas, sep='\n')

In [ ]:
gas_land1 = ['CO2','CH4', 'N2O']
df_L1_gas_sub = df_L1_gas.loc[gas_land1].iloc[:, -4:-1]
df_L1_gas_sub

### Land Use, Land-Use Change, Forestry (LULCF)

In [ ]:
land2_gas_glob = "Chapter 6*/Table 6-2.csv"
title_L2_gas, _, df_L2_gas = parse_report_tables(land2_gas_glob, 32) # The notes are in another column

In [ ]:
metaL2 = pd.read_csv(list(base_path.glob(land2_gas_glob))[0])
notes_L2_gas = metaL2.iloc[32:,:2].values.tolist()
print(title_L2_gas,*notes_L2_gas, sep='\n')

In [ ]:
gas_land2 = ['Carbon Stock Change (CO2)a','CH4', 'N2O']
df_L2_gas_sub = df_L2_gas.loc[gas_land2].iloc[:, -3:]
df_L2_gas_sub = df_L2_gas_sub.rename(index={'Carbon Stock Change (CO2)a':'CO2'})

### Combine L1 and L2

In [ ]:
df_L_gas_sub = df_L1_gas_sub + df_L2_gas_sub
df_L_gas_sub

### Waste Management

In [ ]:
man_was_glob = "Chapter 2*/Table 2-9.csv"
title_W_gas, notes_W_gas, df_W_gas = parse_report_tables(man_was_glob, 10)
print(title_W_gas,*notes_W_gas, sep='\n')

In [ ]:
gas_was = ['CH4', 'N2O']
df_W_gas_sub = df_W_gas.loc[gas_was].iloc[:, -4:-1]
df_W_gas_sub

### Make dictionary of dataframes by emittype

In [ ]:
dataframes_gastype = {
    sectors_final[0]: df_E_gas_sub,
    sectors_final[1]: df_M_gas_sub,
    sectors_final[2]: df_L_gas_sub,
    sectors_final[3]: df_W_gas_sub,
    
}

In [ ]:
# for df in dataframes_gastype.values():
#     # print(df)
#     df["2021-2023 Average"] = df.mean(axis=1)
#     df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100

In [ ]:
# Write to csv:
for key, df in dataframes_gastype.items():
    df["2021-2023 Average"] = df.mean(axis=1)
    if key != sectors_final[2]:
        df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100 # Calculate percent for all but land management
    output=f'output_tables/{key}_Gases.csv'
    df.to_csv(output)

In [ ]:
# dataframes_emittype
# dataframes_gastype

## Plot

In [ ]:
# fig.suptitle('2021-2023 Average Emissions',y=0.95)
for (idx, (sector, df1)), (idx, (sector, df2)) in zip(enumerate(dataframes_emittype.items()), enumerate(dataframes_gastype.items())):
    # Skip land use
    if idx==2:
        continue
    else:
        fig, ax = plt.subplots(1, 2, figsize=(12,10), 
                       constrained_layout=True
                      )
        fig.suptitle(sector, y=0.9)
        
        df1_avg = df1.mean(axis=1).to_frame('2021-2023 Average')
        df1_avg["Percent"] = (df1_avg.iloc[:, 0] / df1_avg.iloc[:, 0].sum()).apply(lambda x: f"{x:.3%}")
        # print(df1_avg)
        ax[0].pie(df1_avg.iloc[:,0], 
                           startangle=90,)
        ax[0].legend(df1_avg.index,
                    # loc="best",
                    )
        ax[0].set_title('Emission Type')

        df2_avg = df2.mean(axis=1).to_frame('2021-2023 Average')
        df2_avg["Percent"] = (df2_avg.iloc[:, 0] / df2_avg.iloc[:, 0].sum()).apply(lambda x: f"{x:.3%}")
        ax[1].pie(df2_avg.iloc[:,0], 
                           startangle=90,)
        ax[1].legend(df2_avg.index)
        
        ax[1].set_title('Gas Type')

                # # axis.legend(labels, 
        # #             fontsize=9,
        # #             bbox_to_anchor=(2, 0.5)
        # #            )
    
        # # labels = [f"{label}: {percent}" for label, percent in zip(df_avg.index, df_avg['Percent'])]
        # # axis = ax.flat[idx]
        # # axis.pie(df_avg.iloc[:,0], 
        # #                    startangle=90,
        # #                   )
        # # axis.legend(labels, 
        # #             loc="center right", 
        # #             fontsize=9,
        # #             bbox_to_anchor=(2, 0.5)$ 
        # #            )
